In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# from keras.preprocessing.image import load_img 
# from keras.preprocessing.image import img_to_array 
# from keras.applications.vgg16 import preprocess_input 

# # models 
# from keras.applications.vgg16 import VGG16 
# from keras.models import Model
# from keras.models import Sequential
# from keras import Input
# from keras import layers
# # clustering and dimension reduction
# from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA

# import tensorflow as tf

# # for everything else
import os
# import numpy as np
# import matplotlib.pyplot as plt
# from random import randint
# import pandas as pd
# import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.preprocessing.image import ImageDataGenerator

import tensorflow_datasets as tfds

In [4]:
path = r"/content/drive/MyDrive/HARP_DATASET/1_day_image/square_images_size224"
os.chdir(path)

spectrograms = []

with os.scandir(path) as files:
    for file in files:
        if file.name.endswith('.png'):
            spectrograms.append(file.name)

print(len(spectrograms))

3269


In [5]:
initial_model = keras.Sequential(
    [
      layers.Rescaling(1./255, input_shape=(256, 256, 3)),
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
    ]
)

feature_extractor = keras.Model(
    inputs=initial_model.inputs,
    outputs=[layer.output for layer in initial_model.layers],
)

# Call feature extractor on test input.
x = tf.ones((1, 256, 256, 3))
features = feature_extractor(x)


In [33]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(path,label_mode=None,batch_size=100, validation_split=0.2, subset="training", seed = 22,image_size=(256, 256))
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(path,label_mode=None,batch_size=100, validation_split=0.2, subset="validation", seed = 22,image_size=(256, 256))

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)


Found 3393 files belonging to 1 classes.
Using 2715 files for training.
Found 3393 files belonging to 1 classes.
Using 678 files for validation.


In [35]:
initial_model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [37]:
epochs = 10
batch_size = 50
history = initial_model.fit(train_dataset, epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=validation_dataset,
                    verbose=1)

Epoch 1/10


ValueError: ignored